<a href="https://colab.research.google.com/github/ShivaniGawande/SalaryEstimator/blob/main/SalaryPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [256]:
#install optuna - framework for hyperparameter optimization
% pip install optuna

In [257]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from math import sqrt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from xgboost import XGBRegressor
import optuna

import warnings
warnings.filterwarnings('ignore')

In [258]:
df = pd.read_csv('https://raw.githubusercontent.com/ShivaniGawande/SalaryEstimator/main/data.csv')
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,tensor,hadoop,tableau,bi,flink,mongo,google_an,job_title_sim,seniority_by_title,Degree
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 - 1000,1973,...,0,0,1,1,0,0,0,data scientist,na,M
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+,1984,...,0,0,0,0,0,0,0,data scientist,na,M
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 - 1000,2010,...,0,0,0,0,0,0,0,data scientist,na,M
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 - 5000,1965,...,0,0,0,0,0,0,0,data scientist,na,na
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 - 200,1998,...,0,0,0,0,0,0,0,data scientist,na,na


In [259]:
def approach1_XGBRegressor():
    xgb = XGBRegressor()
    parameters = {'learning_rate': [.01, 0.1, .03], 
              'max_depth': [5, 7, 9],
              'n_estimators': [100,200,300]}

    xgb_grid = GridSearchCV(xgb, parameters, cv = 2, n_jobs = 5, verbose=True)
    return xgb_grid

In [260]:
def approach2_SVRRegressor():
    #SVR(kernel="rbf", C=100, gamma=0.1, epsilon=0.1)
    svr = SVR()
    parameters = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf']}

    svr_grid = GridSearchCV(svr, parameters, cv=2, n_jobs = 5, verbose=True)
    return svr_grid


In [261]:

def approach3_RandomForestRegressor():
    #RandomForestRegressor(max_depth=2, random_state=0)
    rfr= RandomForestRegressor()
    parameters = {'max_depth': [40, 50, 60],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [4, 5, 6],
    'n_estimators': [150, 200]}

    rfr_grid = GridSearchCV(rfr, parameters, cv=2, n_jobs = 5, verbose=True)
    return rfr_grid

In [262]:
def feature_importance(model,x):
    df = pd.DataFrame(model.feature_importances_,columns=['importance'])
    df['features'] = x.columns
    df.reindex(x.columns, axis=1)

def model_accuracy(model, xtrain, xtest, ytrain, ytest):
    #evaluate
    scores = cross_val_score(model, xtrain, ytrain,cv=5)
    print("Mean cross-validation score: %.2f" % scores.mean())
    ypred = model.predict(xtest)
    mse = mean_squared_error(ytest, ypred)
    rmse = sqrt(mse)
    print("RMSE: %.2f" % rmse)
    y_pred_test = model.predict(xtest)

def train_model(x,y, model):
    xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.2)
    model.fit(xtrain, ytrain)
    #print(" Best Params : ", model.best_params_)
    #print(xtest)
    return model, xtrain, xtest, ytrain, ytest

In [ ]:
Y = df.iloc[:,19]
X = pd.concat([df.iloc[:,4],df.iloc[:,23:39],df.iloc[:,21],df.iloc[:,8],df.iloc[:,10],df.iloc[:,11],df.iloc[:,12],df.iloc[:,40:42]],axis = 1)
X["Job Location"] = X["Job Location"].astype("category")
X["Industry"]=X["Industry"].astype("category")
X["Sector"] = X["Sector"].astype("category")
X["seniority_by_title"] = X["seniority_by_title"].astype("category")
X["Degree"] = X["Degree"].astype("category")
X["Size"] = X["Size"].astype("category")
X["Type of ownership"]=X["Type of ownership"].astype("category")

X["Job Location"] = X["Job Location"].cat.codes
X["Industry"] = X["Industry"].cat.codes
X["Sector"] = X["Sector"].cat.codes
X["seniority_by_title"] = X["seniority_by_title"].cat.codes
X["Degree"] = X["Degree"].cat.codes
X["Size"] = X["Size"].cat.codes
X["Type of ownership"] = X["Type of ownership"].cat.codes

print("Approach 1: Using XGB")
model = approach1_XGBRegressor()
model, xtrain, xtest, ytrain, ytest  = train_model(X,Y, model)
print("Model Accuracy on Test Dataset")
model_accuracy(model, xtrain, xtest, ytrain, ytest)
#print(X.columns)


print("Approach 2: Using SVR")
model = approach2_SVRRegressor()
model, xtrain, xtest, ytrain, ytest  = train_model(X,Y, model)
print("Model Accuracy on Test Dataset")
model_accuracy(model, xtrain, xtest, ytrain, ytest)
#print(X.columns)


print("Approach 3: Using Random Forest Regressor")
model = approach3_RandomForestRegressor()
model, xtrain, xtest, ytrain, ytest  = train_model(X,Y, model)
print("Model Accuracy on Test Dataset")
model_accuracy(model, xtrain, xtest, ytrain, ytest)
#print(X.columns)

Approach 1: Using XGB
Fitting 2 folds for each of 27 candidates, totalling 54 fits
[23:29:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Model Accuracy on Test Dataset
Fitting 2 folds for each of 27 candidates, totalling 54 fits
[23:30:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 2 folds for each of 27 candidates, totalling 54 fits
[23:30:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 2 folds for each of 27 candidates, totalling 54 fits
[23:30:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 2 folds for each of 27 candidates, totalling 54 fits
[23:30:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting

In [ ]:
xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.2)

In [ ]:
def objective_XGB(trial):
   learning_rate = trial.suggest_float("learning_rate", 1e-4, 0.25, log=True)
   reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-9, 100.0)
   reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-9, 100.0)
   subsample = trial.suggest_float("subsample", 0.0001, 1.0)
   colsample_bytree = trial.suggest_float("colsample_bytree", 0.0001, 1.0)
   max_depth = trial.suggest_int("max_depth", 1, 50)
   early_stopping_rounds = trial.suggest_int("early_stopping_rounds", 100, 700)
   n_estimators = trial.suggest_int("n_estimators", 0, 10000)

   model = XGBRegressor(
      random_state=50,
      n_estimators=n_estimators,
      learning_rate=learning_rate,
      reg_lambda=reg_lambda,
      reg_alpha=reg_alpha,
      subsample=subsample,
      colsample_bytree=colsample_bytree,
      max_depth=max_depth,
   )
   model.fit(xtrain,ytrain,
      early_stopping_rounds=early_stopping_rounds,
      eval_set=[(xtest,ytest)],
      verbose=1200
   )
   preds_valid = model.predict(xtest)
   rmse = mean_squared_error(ytest, preds_valid, squared=False)

   return rmse



In [ ]:
study_xgb = optuna.create_study(direction="minimize")
study_xgb.optimize(objective_XGB, n_trials=10)

optuna_params_xgb = study_xgb.best_params
optuna_params_xgb

In [ ]:
print("Approach 1: Using XGBoost ")
model = XGBRegressor( early_stopping_rounds=optuna_params_xgb['early_stopping_rounds'],
      n_estimators=optuna_params_xgb['n_estimators'],
      learning_rate=optuna_params_xgb['learning_rate'],
      reg_lambda=optuna_params_xgb['reg_lambda'],
      reg_alpha=optuna_params_xgb['reg_alpha'],
      subsample=optuna_params_xgb['subsample'],
      colsample_bytree=optuna_params_xgb['colsample_bytree'],
      max_depth=optuna_params_xgb['max_depth'],)
model, xtrain, xtest, ytrain, ytest  = train_model(X,Y, model)
#feature_importance(model,X)
print("Model Accuracy on Test Dataset")
model_accuracy(model, xtrain, xtest, ytrain, ytest)
#print(X.columns)
